# Meteo con el pato

### Este informe muestra los datos diarios de temperatura recogidos en el portal de datos abiertos del ayuntamiento.
* dataset: **meteorologia-tiempo-real** 

In [33]:
%%capture
import duckdb
import hvplot.duckdb


%load_ext sql
conn = duckdb.connect() #duckdb.connect('/home/jcaubin/duck_test.db') #conexion a bbdd en memoria o en fichero
%sql conn --alias duckdb

In [34]:
%%capture
%%sql 

create table meteo_today as
select * from read_csv('https://datos.madrid.es/egob/catalogo/300392-11041819-meteorologia-tiempo-real.csv');

In [35]:
%%capture
%%sql

create or replace table td as
select PROVINCIA, MUNICIPIO, ESTACION, MAGNITUD, PUNTO_MUESTREO,ANO, MES, DIA, cast(right(HORA,2) as int) H, VALOR	 from (
from meteo_today
unpivot (
    (VALOR)
for HORA in (
    H01, 
    H02, 
    H03, 
    H04, 
    H05, 
    H06, 
    H07, 
    H08, 
    H09, 
    H10, 
    H11, 
    H12, 
    H13, 
    H14, 
    H15, 
    H16, 
    H17, 
    H18, 
    H19, 
    H20, 
    H21, 
    H22, 
    H23, 
    H24 
    ) 
)
);

In [36]:
%%capture
%%sql 

create or replace  table tv as
select PROVINCIA, MUNICIPIO, ESTACION, MAGNITUD, PUNTO_MUESTREO,ANO, MES, DIA, cast(right(HORA,2) as int) H, VALIDEZ	 
from (
from meteo_today
unpivot (
    (VALIDEZ)
for HORA in (
    V01, 
    V02, 
    V03, 
    V04, 
    V05, 
    V06, 
    V07, 
    V08, 
    V09, 
    V10, 
    V11, 
    V12, 
    V13, 
    V14, 
    V15, 
    V16, 
    V17, 
    V18, 
    V19, 
    V20, 
    V21, 
    V22, 
    V23, 
    V24, 
    ) 
 ))
 ;


In [37]:
%%capture
%%sql

create or replace table meteo_madrid24 as
select td.*, tv.VALIDEZ
from td 
inner join tv on td.PROVINCIA=tv.PROVINCIA and 	td.MUNICIPIO=tv.MUNICIPIO and td.ESTACION=tv.ESTACION and 	td.MAGNITUD = tv.MAGNITUD
    and 	td.PUNTO_MUESTREO = tv.PUNTO_MUESTREO and td.ANO= tv.ANO and td.MES = tv.MES and TD.DIA = tv.DIA and td.H=tv.H

In [ ]:

relation = conn.sql("""PIVOT(
select estacion, h, valor
from meteo_madrid24
where
magnitud = 83 --temperatura
and validez = 'V'
)
on ESTACION
USING sum(VALOR)
group by h""")


In [ ]:
import panel as pn
import hvplot.duckdb

pn.extension() # for notebook
pn.Column(relation.hvplot.line(y=['8', '58', '24', '107', '102'], x='h'))

In [ ]:
#imprime la hora de ejecución
from datetime import datetime
from IPython.display import display, Markdown

t = datetime.now()
display(Markdown(f'### hora de ejecución: {t}'))